In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import cv2

import numpy as np
from collections import defaultdict


from PIL import Image, ImageFilter
import io
import re
import random
import numpy.random as npr
from skimage import data
from scipy.ndimage import rotate
from kernels import *
import torchvision
from torchvision import models
import os
from torchvision.transforms.functional import to_pil_image
from torch.utils.data import Dataset, DataLoader, Subset

import torchvision.transforms as transforms
 
import my_utils as ut
import old_utils_averaged_filters as old_ut_avg
import old_utils_multi_image_old_snr as old_ut_multi
from transformers import Swinv2ForImageClassification, SwinConfig
from torch.optim import AdamW
from torchvision import transforms, datasets



In [3]:

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
   
])

# Create dataset instances
train_dataset = ut.DatasetAI(root_dir='/mnt/e/GenImage', transform=transform, split='train')
val_test_dataset = ut.DatasetAI(root_dir='/mnt/e/GenImage', transform=transform, split='val')


train_subset,val_subset, test_subset = ut.split_datasets(train_dataset, val_test_dataset, 30000, 3750, 3750)






train_loader = DataLoader(train_subset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_subset, batch_size=32, shuffle=False, num_workers=4)
test_loader = DataLoader(test_subset, batch_size=32, shuffle=False, num_workers=4)



ValueError: num_samples should be a positive integer value, but got num_samples=0

In [4]:
#check distribution of classes 

print("Check overlap between datasets:\n")
print("\nTrain Subset and Validation Subset:")
ut.check_data_overlap(train_subset, val_subset)
print("\nTrain Subset and Test Subset:")
ut.check_data_overlap(train_subset, test_subset)
print("\nValidation Subset and Test Subset:")
ut.check_data_overlap(val_subset, test_subset)

print()
print("\nTrain Subset Distribution:\n")
ut.print_model_class_distribution(train_dataset, train_subset.indices)
print("\nValidation Subset Distribution:\n")
ut.print_model_class_distribution(val_test_dataset, val_subset.indices)
print("\nTest Subset Distribution:\n")
ut.print_model_class_distribution(val_test_dataset, test_subset.indices)

Check overlap between datasets:


Train Subset and Validation Subset:
No actual data overlap detected.

Train Subset and Test Subset:
No actual data overlap detected.

Validation Subset and Test Subset:
No actual data overlap detected.


Train Subset Distribution:

Total samples in subset: 8
Model ADM, Class nature: 1 (12.50%)
Model BigGAN, Class nature: 1 (12.50%)
Model Midjourney, Class nature: 1 (12.50%)
Model VQDM, Class nature: 1 (12.50%)
Model glide, Class ai: 1 (12.50%)
Model stable_diffusion_v_1_4, Class nature: 1 (12.50%)
Model stable_diffusion_v_1_5, Class nature: 1 (12.50%)
Model wukong, Class nature: 1 (12.50%)

Validation Subset Distribution:

Total samples in subset: 8
Model ADM, Class nature: 1 (12.50%)
Model BigGAN, Class nature: 1 (12.50%)
Model Midjourney, Class nature: 1 (12.50%)
Model VQDM, Class ai: 1 (12.50%)
Model glide, Class nature: 1 (12.50%)
Model stable_diffusion_v_1_4, Class nature: 1 (12.50%)
Model stable_diffusion_v_1_5, Class nature: 1 (12.50%)
Model wuk

In [5]:
class BaseClassifier(nn.Module):
    def __init__(self,kernels):
        super(BaseClassifier, self).__init__()
        self.feature_combiner = ut.CNNBlock(kernels)
        self.feature_combiner2 = ut.CNNBlock(kernels)
        self.classifier = ut.DeepClassifier() 

 
    def forward(self, rich, poor):
       
        x = self.feature_combiner(rich)
        y = self.feature_combiner2(poor)   
        feature_difference = x - y
        outputs = self.classifier(feature_difference)

        return outputs

In [6]:
class ResNetClassifier(nn.Module):
    def __init__(self,kernels):
        super(ResNetClassifier, self).__init__()
        self.feature_combiner = ut.CNNBlock(kernels)
        self.feature_combiner2 = ut.CNNBlock(kernels)
        resnet_weights = models.ResNet50_Weights.DEFAULT
        self.resnet = models.resnet50(weights=resnet_weights)
        # Remove the final fully connected layer
        self.features = nn.Sequential(*list(self.resnet.children())[:-2])
        # Add a new classifier layer
        self.classifier = nn.Linear(self.resnet.fc.in_features, 2)
        # Adaptive pool to make sure output from feature maps is of fixed size
        self.adaptive_pool = nn.AdaptiveAvgPool2d((1, 1))

 
    def forward(self, rich, poor):
       
        x = self.feature_combiner(rich)
        y = self.feature_combiner2(poor)   
        feature_difference = x - y
        
        # Process feature difference through the ResNet features
        features = self.features(feature_difference)
        pooled_features = self.adaptive_pool(features)
        flat_features = torch.flatten(pooled_features, 1)
        outputs = self.classifier(flat_features)

        return outputs

In [7]:
class SwinClassification(nn.Module):
    def __init__(self,kernels):
        super(SwinClassification, self).__init__()
        self.feature_combiner = ut.CNNBlock(kernels)
        self.feature_combiner2 = ut.CNNBlock(kernels)
        config = SwinConfig.from_pretrained('microsoft/swinv2-tiny-patch4-window8-256',num_classes=2)
        self.transformer = Swinv2ForImageClassification.from_pretrained(
            "microsoft/swinv2-tiny-patch4-window8-256",
            config=config
        )
        
        self.transformer.classifier = nn.Linear(config.hidden_size, 2) 

 
    def forward(self, rich, poor):
       
        x = self.feature_combiner(rich)
        y = self.feature_combiner2(poor)   
        feature_difference = x - y
        outputs = self.transformer(feature_difference)

        return outputs.logits


## Train with freezing the pretrained models and only train the last layer with new preprocessing

In [ ]:
#Swin Transformer
learning_rates = [1e-3,1e-4, 1e-5 ]
#print model and freeze and pre
print("Training SWIN Transformer Frozen w/New Pre\n")

for learning_rate in learning_rates: 
    print(f"Training with Learning Rate for SWIN: {learning_rate} \n\n")   
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    kernels =  ut.apply_high_pass_filter()
    model = SwinClassification(kernels).to(device)
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    best_model_path = '/home/kosta/code/School/SentryAI/pth/best_model_newPre_SWIN_frozen.pth'

    #freeze the transformer layers
    for param in model.transformer.parameters():
        param.requires_grad = False
    #unfreeze the classifier layer
    for param in model.transformer.classifier.parameters():
        param.requires_grad = True

    ut.train_and_validate(model, train_loader, val_loader, optimizer, device, num_epochs=10, best_model_path= best_model_path)



In [ ]:
#ResNET
print("Training RES Frozen w/New Pre\n")
learning_rates = [1e-3,1e-4, 1e-5 ]
for learning_rate in learning_rates:
    print(f"Training with Learning Rate: {learning_rate} \n\n")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    kernels =  ut.apply_high_pass_filter()
    model = ResNetClassifier(kernels).to(device)
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    best_model_path = '/home/kosta/code/School/SentryAI/pth/best_model_newPre_RESNET_frozen.pth'

    #freeze resnet layers
    for param in model.resnet.parameters():
        param.requires_grad = False
    #unfreeze the classifier layer
    for param in model.classifier.parameters():
        param.requires_grad = True
    ut.train_and_validate(model, train_loader, val_loader, optimizer, device, num_epochs=10, best_model_path= best_model_path)



## Unfreeze the pretrained model and Original Model with new preprocessing

In [ ]:
#Swin Transformer
learning_rates = [1e-3,1e-4, 1e-5, 1e-6]
print("Training SWIN Transformer UNFrozen w/New Pre\n")
for learning_rate in learning_rates:
    print(f"Training with Learning Rate: {learning_rate} \n\n")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    kernels =  ut.apply_high_pass_filter()
    model = SwinClassification(kernels).to(device)
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    best_model_path = '/home/kosta/code/School/SentryAI/pth/best_model_newPre_SWIN_unfrozen.pth'



    ut.train_and_validate(model, train_loader, val_loader, optimizer, device, num_epochs=10, best_model_path= best_model_path)



In [10]:
#ResNET
print("Training RES UNFrozen w/New Pre\n")

for learning_rate in learning_rates:
    
    print(f"Training with Learning Rate: {learning_rate} \n\n")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    kernels =  ut.apply_high_pass_filter()
    model = ResNetClassifier(kernels).to(device)
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    best_model_path = '/home/kosta/code/School/SentryAI/pth/best_model_newPre_RESNET_unfrozen.pth'


    ut.train_and_validate(model, train_loader, val_loader, optimizer, device, num_epochs=10, best_model_path= best_model_path)



Training with Learning Rate: 0.001 


Loaded previous best model with accuracy: 0.875
Epoch 1/10
,Train Loss: 0.7449, Train Acc: 0.2500, Val Loss: 0.5537, Val Acc: 0.7500

Epoch 2/10
,Train Loss: 0.3480, Train Acc: 0.8750, Val Loss: 0.5469, Val Acc: 0.7500

Epoch 3/10
,Train Loss: 0.0875, Train Acc: 1.0000, Val Loss: 0.5909, Val Acc: 0.7500

Epoch 4/10
,Train Loss: 0.0261, Train Acc: 1.0000, Val Loss: 0.5138, Val Acc: 0.7500

Epoch 5/10
,Train Loss: 0.0037, Train Acc: 1.0000, Val Loss: 0.4869, Val Acc: 0.7500

Epoch 6/10
,Train Loss: 0.0009, Train Acc: 1.0000, Val Loss: 0.4863, Val Acc: 0.7500

Epoch 7/10
,Train Loss: 0.0003, Train Acc: 1.0000, Val Loss: 0.5310, Val Acc: 0.7500

Epoch 8/10
,Train Loss: 0.0002, Train Acc: 1.0000, Val Loss: 0.5841, Val Acc: 0.7500

Epoch 9/10
,Train Loss: 0.0001, Train Acc: 1.0000, Val Loss: 0.6378, Val Acc: 0.7500

Epoch 10/10
,Train Loss: 0.0001, Train Acc: 1.0000, Val Loss: 0.6231, Val Acc: 0.7500

Training with Learning Rate: 0.0001 


Loaded previou

In [ ]:
#original model
print("Training BASE w/New Pre\n")

for learning_rate in learning_rates:
    
    print(f"Training with Learning Rate: {learning_rate} \n\n")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    kernels =  ut.apply_high_pass_filter()
    model = BaseClassifier(kernels).to(device)
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    best_model_path = '/home/kosta/code/School/SentryAI/pth/best_model_newPre_Base.pth'
    
    
    ut.train_and_validate(model, train_loader, val_loader, optimizer, device, num_epochs=10, best_model_path= best_model_path)

